In [1]:
import pandas as pd
import sys
sys.path.append('/home/greg/GitHub/')
import pyEDIutils.search as edi

## Get table from EDI

This lists all data packages at EDI under the knb-lter-jrn scope.

In [2]:
df = edi.search_pasta(fields=['packageid','title','pubdate','keyword','author',
                                   'begindate','enddate','doi'])
df.head()

https://pasta.lternet.edu/package/search/eml?defType=edismax&q=%2A&fq=scope%3Aknb-lter-jrn&fl=packageid%2Ctitle%2Cpubdate%2Ckeyword%2Cauthor%2Cbegindate%2Cenddate%2Cdoi&sort=packageid%2Casc&rows=500


,packageid,title,pubdate,keywords,authors,begindate,enddate,doi
0,knb-lter-jrn.100.3,High resolution shrub cover raster maps of the...,2020,canopy cover;land surface properties;plant cov...,"Ji, Wenjie;Hanan, Niall P.",2011-01-01,2011-12-31,doi:10.6073/pasta/313fec8669bc7b4d8debf7393dd2...
1,knb-lter-jrn.101.1,Thresholds and alternative states in neotropic...,2021,Disturbance;Vegetation Dynamics;plant communit...,"Peinetti, H. Raúl;Bestelmeyer, Brandon;Chirino...",2005-09-01,2018-03-31,doi:10.6073/pasta/e1978467bd09fc4f6e0cd8509b06...
2,knb-lter-jrn.102.1,Carbon exchange responses of rehydrated and in...,2023,populations;algae;bacterial abundance;carbon a...,"Hoellrich, Mikaela;Pietrasiak, Nicole",2020-01-01,2022-12-31,doi:10.6073/pasta/c6ffd88dc80df1ed1ec32ccdc477...
3,knb-lter-jrn.103.1,Gridded 1-hectare estimates of shrub community...,2023,JRN LTER;Jornada Basin;LiDAR;aerial imagery;pl...,"Roberts, Trevor H;Hanan, Niall P",2011-01-01,2019-12-31,doi:10.6073/pasta/49bec3ac2917e494c5de4b19130e...
4,knb-lter-jrn.200021001.5,Data to explore circular manureshed management...,2023,crop-livestock integration;manureshed;LTAR;ani...,"Spiegal, Sheri;Vendramini, João M.B.;Bittman, ...",2010-01-01,2019-12-31,doi:10.6073/pasta/6025924b89e313f25bddf1912c8a...


In [5]:
# Extract package ID
pkgid = df.packageid.str.split('.', expand=True).loc[:,1]
df['pkgid'] = pkgid.astype(str)
# Series ids (prj, jer, other)
df['seriesid'] = 'other'
df.loc[pkgid.str[-9:-6]=='200', 'seriesid'] = 'jer'
df.loc[pkgid.str[-9:-6]=='210', 'seriesid'] = 'prj'
# Study IDs
df['studyid'] = pkgid.astype(str).str[0:-2]
df.loc[pkgid.astype(int) > 200000000, 'studyid'] = pkgid.loc[pkgid.astype(int) > 200000000].str[-6:-3]
df['studyid2'] = df['seriesid'] + df['studyid']
df.head()

,packageid,title,pubdate,keywords,authors,begindate,enddate,doi,pkgid,seriesid,studyid,studyid2
0,knb-lter-jrn.100.3,High resolution shrub cover raster maps of the...,2020,canopy cover;land surface properties;plant cov...,"Ji, Wenjie;Hanan, Niall P.",2011-01-01,2011-12-31,doi:10.6073/pasta/313fec8669bc7b4d8debf7393dd2...,100,other,1,other1
1,knb-lter-jrn.101.1,Thresholds and alternative states in neotropic...,2021,Disturbance;Vegetation Dynamics;plant communit...,"Peinetti, H. Raúl;Bestelmeyer, Brandon;Chirino...",2005-09-01,2018-03-31,doi:10.6073/pasta/e1978467bd09fc4f6e0cd8509b06...,101,other,1,other1
2,knb-lter-jrn.102.1,Carbon exchange responses of rehydrated and in...,2023,populations;algae;bacterial abundance;carbon a...,"Hoellrich, Mikaela;Pietrasiak, Nicole",2020-01-01,2022-12-31,doi:10.6073/pasta/c6ffd88dc80df1ed1ec32ccdc477...,102,other,1,other1
3,knb-lter-jrn.103.1,Gridded 1-hectare estimates of shrub community...,2023,JRN LTER;Jornada Basin;LiDAR;aerial imagery;pl...,"Roberts, Trevor H;Hanan, Niall P",2011-01-01,2019-12-31,doi:10.6073/pasta/49bec3ac2917e494c5de4b19130e...,103,other,1,other1
4,knb-lter-jrn.200021001.5,Data to explore circular manureshed management...,2023,crop-livestock integration;manureshed;LTAR;ani...,"Spiegal, Sheri;Vendramini, João M.B.;Bittman, ...",2010-01-01,2019-12-31,doi:10.6073/pasta/6025924b89e313f25bddf1912c8a...,200021001,jer,021,jer021


## Read local status of packages

This table is where we assign core, signature, local keywords, and other categorizations

In [6]:
localstatus = df.loc[:, ['seriesid', 'studyid', 'studyid2', 'pkgid', 'pubdate', 'title', 'authors']]
dtype_dic= { 'studyid': str }
#localstatus = pd.read_csv('edi_package_localstatus.csv', converters={'studyid': lambda x: str(x)})
old_localstatus = pd.read_csv('edi_package_localstatus_jrn7.csv', dtype = dtype_dic)
print(len(localstatus))
print(len(old_localstatus))
print(localstatus.info())
print(old_localstatus.info())

388
211
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 388 entries, 0 to 387
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   seriesid  388 non-null    object
 1   studyid   388 non-null    object
 2   studyid2  388 non-null    object
 3   pkgid     388 non-null    object
 4   pubdate   388 non-null    object
 5   title     388 non-null    object
 6   authors   388 non-null    object
dtypes: object(7)
memory usage: 21.3+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 211 entries, 0 to 210
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   studyid            211 non-null    object 
 1   pkgid              211 non-null    int64  
 2   pubdate            211 non-null    int64  
 3   title              211 non-null    object 
 4   authors            211 non-null    object 
 5   old_core           61 non-null     object 
 6   core_s

### Make changes if needed and resave

Add keywords, alter core/signature, etc

In [7]:
print(localstatus.studyid.unique())
sum(localstatus.studyid.isna())

['1' '021' '022' '029' '030' '034' '043' '045' '046' '047' '048' '2' '001'
 '002' '007' '008' '009' '010' '011' '012' '013' '015' '086' '119' '120'
 '121' '122' '126' '127' '128' '228' '259' '262' '278' '287' '288' '301'
 '308' '313' '327' '328' '338' '349' '351' '365' '368' '369' '370' '379'
 '380' '385' '386' '389' '392' '394' '395' '398' '407' '408' '409' '412'
 '413' '420' '425' '437' '444' '461' '465' '469' '472' '493' '500' '520'
 '527' '539' '543' '548' '574' '586']


0

In [8]:
#Assign core areas by study id
localstatus['corearea'] = 'None'
lulc = localstatus.studyid2.isin(['other1','prj119','prj120','prj392','prj413','prj472', 'prj543',
    'jer021','jer022','jer029','jer030','jer034','jer043','jer045','jer046','jer047','jer048'])
localstatus.loc[lulc, 'corearea'] = 'Land Use and Land Cover Change'

inorganic = localstatus.studyid2.isin(['prj001','prj012','prj013','prj015','prj128',
    'prj228','prj288','prj338','prj365','prj368','prj386','prj394','prj395', 'prj398','prj586'])
localstatus.loc[inorganic, 'corearea'] = 'Inorganic matter cycling'

organic = localstatus.studyid2.isin(['prj301','prj327','prj389'])
localstatus.loc[organic, 'corearea'] = 'Organic matter cycling'

pops = localstatus.studyid2.isin(['prj007','prj008', 'prj009', 'prj010','prj351','prj370',
    'prj369','prj385','prj412','prj444','prj469','prj493','prj500','prj520'])
localstatus.loc[pops, 'corearea'] = 'Populations'

disturbance = localstatus.studyid2.isin(['prj002','prj122','prj126','prj127', 'prj259',
    'prj278','prj308','prj313','prj328','prj379','prj380','prj407','prj425','prj437','prj461',
    'prj465','prj527','prj539','prj548','other2'])
localstatus.loc[disturbance, 'corearea'] = 'Disturbance'

prod = localstatus.studyid2.isin(['prj011','prj287','prj408','prj409','prj574'])
localstatus.loc[prod, 'corearea'] = 'Primary production'

In [9]:
# Assign core areas by pkgid (studies 015, 086, 121, 262)
#lulc = localstatus.pkgid.isna()
#localstatus.loc[lulc, 'corearea'] = 'Land Use and Land Cover Change'

inorganic = localstatus.pkgid.isin(['210015002','210015003', '210420002'])
localstatus.loc[inorganic, 'corearea'] = 'Inorganic matter cycling'

organic = localstatus.pkgid.isin(['210086004', '210420001'])
localstatus.loc[organic, 'corearea'] = 'Organic matter cycling'

pops = localstatus.pkgid.isin(['210086001','210086002','210086003','210086005','210086006','210086008','210086009',
'210121001','210121003', '210121004', '210121006','210262001','210262008','210262010'])
localstatus.loc[pops, 'corearea'] = 'Populations'

disturbance = localstatus.pkgid.isin(['210086007','210121005','210121008','210349002'])
localstatus.loc[disturbance, 'corearea'] = 'Disturbance'

prod = localstatus.pkgid.isin(['210015001','210262004','210262005','210349001', '210349003'])
localstatus.loc[prod, 'corearea'] = 'Primary production'

In [10]:
# Assign core/non-core
localstatus['status'] = ''
study_core = localstatus.studyid2.isin(['prj002','prj011','prj012','prj013','prj126','prj228','prj262','prj287','prj288',
    'prj338','prj349','prj351','prj412','prj413','prj425','prj520','prj444','prj461','prj465','prj574','prj437','prj548',
    'jer043','jer047','jer048'])
localstatus.loc[study_core, 'status'] = 'core'
pkg_core = localstatus.pkgid.isin(['210086001','210086002','210086006','210086009', '210121005','101','103','201','202'])
localstatus.loc[pkg_core, 'status'] = 'core'

localstatus['top10'] = ''
top10ds = localstatus.pkgid.isin(['210351001','210351004','210379001','210574001','210574002', '210126001'
'210539001','210259001','210288001','210308005','210308004','210262005','210262004','210262008','210338006',
'210338002','100'])
localstatus.loc[top10ds, 'top10'] = 'Y'



In [11]:
localstatus.head()

,seriesid,studyid,studyid2,pkgid,pubdate,title,authors,corearea,status,top10
0,other,1,other1,100,2020,High resolution shrub cover raster maps of the...,"Ji, Wenjie;Hanan, Niall P.",Land Use and Land Cover Change,,Y
1,other,1,other1,101,2021,Thresholds and alternative states in neotropic...,"Peinetti, H. Raúl;Bestelmeyer, Brandon;Chirino...",Land Use and Land Cover Change,core,
2,other,1,other1,102,2023,Carbon exchange responses of rehydrated and in...,"Hoellrich, Mikaela;Pietrasiak, Nicole",Land Use and Land Cover Change,,
3,other,1,other1,103,2023,Gridded 1-hectare estimates of shrub community...,"Roberts, Trevor H;Hanan, Niall P",Land Use and Land Cover Change,core,
4,jer,021,jer021,200021001,2023,Data to explore circular manureshed management...,"Spiegal, Sheri;Vendramini, João M.B.;Bittman, ...",Land Use and Land Cover Change,,


In [12]:
localstatus.loc[localstatus.corearea=='None',:]

,seriesid,studyid,studyid2,pkgid,pubdate,title,authors,corearea,status,top10


In [13]:
localstatus_out = localstatus.copy()
localstatus_out.to_csv('edi_package_localstatus_jrn8.csv', index=False)

## Make Table1


In [14]:
# Merge in core designations and rename them to "signature"
t1 = pd.merge(df, localstatus.loc[:,['pkgid','status','corearea',
                                     'top10']], on='pkgid')
#t1.core_sig_2020prop.replace('core', 'signature',inplace=True)
print(len(t1))

# Replace or add whitespace for a couple columns
t1.title = t1.title.str.replace('\n +', ' ')
t1.authors = t1.authors.str.replace(';', '; ')
# Trim version # from package id
t1.packageid = t1.packageid.map(lambda x: x.rsplit('.',1)[0])
# Add a date range
t1['drange'] = t1.begindate + ' to ' + t1.enddate
# Replace core areas with abbreviations
t1.corearea.replace('Disturbance', 'D', inplace=True)
t1.corearea.replace('Land Use and Land Cover Change', 'Lu', inplace=True)
t1.corearea.replace('Primary production', 'Pp', inplace=True)
t1.corearea.replace('Populations', 'Po', inplace=True)
t1.corearea.replace('Inorganic matter cycling', 'Im', inplace=True)
t1.corearea.replace('Organic matter cycling', 'Om', inplace=True)
# Add sorting order column for corearea
t1['corearea_sort'] = 0
t1.corearea_sort.loc[t1.corearea=='Pp'] = 1
t1.corearea_sort.loc[t1.corearea=='Om'] = 2
t1.corearea_sort.loc[t1.corearea=='D'] = 3
t1.corearea_sort.loc[t1.corearea=='Im'] = 4
t1.corearea_sort.loc[t1.corearea=='Po'] = 5
t1.corearea_sort.loc[t1.corearea=='Lu'] = 6
# Sort values
t1 = t1.sort_values(['status', 'corearea_sort', 'pkgid'],
                   ascending=[False, True, True])

# Fill in NA values with empty string
t1['status'].fillna('', inplace=True)
t1['top10'].fillna('', inplace=True)

t1.index = t1.loc[:,'pkgid']
t1.head()

388


/tmp/ipykernel_374715/2683925567.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t1.corearea_sort.loc[t1.corearea=='Pp'] = 1
/tmp/ipykernel_374715/2683925567.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t1.corearea_sort.loc[t1.corearea=='Om'] = 2
/tmp/ipykernel_374715/2683925567.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t1.corearea_sort.loc[t1.corearea=='D'] = 3
/tmp/ipykernel_374715/2683925567.py:26: SettingWit

,packageid,title,pubdate,keywords,authors,begindate,enddate,doi,pkgid,seriesid,studyid,studyid2,status,corearea,top10,drange,corearea_sort
pkgid,,,,,,,,,,,,,,,,,
210011001,knb-lter-jrn.210011001,Seasonal aboveground plant biomass estimates a...,2023,populations;primary production;aboveground pro...,"Peters, Debra C; Huenneke, Laura F",1989-05-01,2022-10-31,doi:10.6073/pasta/5507ade25993c2123e1886a545b5...,210011001,prj,011,prj011,core,Pp,,1989-05-01 to 2022-10-31,1
210011002,knb-lter-jrn.210011002,Seasonal non-destructive vegetation measuremen...,2023,populations;primary production;aboveground pro...,"Peters, Debra C; Huenneke, Laura F",1989-05-01,2022-10-31,doi:10.6073/pasta/4fb9b7b5ec3d0692b280bd74e8b0...,210011002,prj,011,prj011,core,Pp,,1989-05-01 to 2022-10-31,1
210011003,knb-lter-jrn.210011003,Annual mean estimates of aboveground net prima...,2023,populations;primary production;aboveground pro...,"Peters, Debra C; Huenneke, Laura F",1990-01-01,2022-12-31,doi:10.6073/pasta/70d2a2b922bd629b72cf7271bb77...,210011003,prj,011,prj011,core,Pp,,1990-01-01 to 2022-12-31,1
210011004,knb-lter-jrn.210011004,Seasonal reference harvest measurements of veg...,2023,populations;primary production;aboveground pro...,"Peters, Debra C; Huenneke, Laura F",1989-04-25,2023-05-15,doi:10.6073/pasta/f80e80f3501d000bf81c426362cb...,210011004,prj,011,prj011,core,Pp,,1989-04-25 to 2023-05-15,1
210011005,knb-lter-jrn.210011005,Annual ground-based photographs taken at 15 ne...,2019,net primary production;aboveground production;...,"Anderson, John",2003-10-14,2018-09-18,doi:10.6073/pasta/69ca7ac7254a2b236d3f44b21fa8...,210011005,prj,011,prj011,core,Pp,,2003-10-14 to 2018-09-18,1


## Special cases (on Deb's orders)

In [79]:
t1.authors.loc[t1.pkgid.isin([210011001, 210011002, 210011003, 210011004, 210287001])] = 'Peters, Debra; Huenneke, Laura'
t1.authors.loc[t1.pkgid.isin([210002001])] = 'Peters, Debra; Anderson, John; Huenneke, Laura'
t1.authors.loc[t1.pkgid.isin([210121005])] = 'Peters, Debra; Anderson, John; Rango, Al; Huenneke, Laura'
t1.authors.loc[t1.pkgid.isin([210308005])] = 'Peters, Debra; Herrick, Jeff; Okin, Gregory S'
t1.authors.loc[t1.pkgid.isin([210351003])] = 'Bestelmeyer, Brandon; Yao, Jin'
# Append symbol to SMES and similar packages (multiyear=asterisk or complete=dagger)
multiyr = [210086002,210086004,210086005,210086007,210120001,210120002]
t1.title.loc[t1.pkgid.isin(multiyr)] = t1.title.loc[t1.pkgid.isin(multiyr)] + '*'
#complete = [210086001,210086003,210086008,210086009]
#t1.title.loc[t1.pkgid.isin(complete)] = t1.title.loc[t1.pkgid.isin(complete)] + u'\u2020'

# Just drop the most useful NPP package from the table (Sigh)
# and the deprecated soil water
t1 = t1.drop([210011001,210013002])
t1.head()

/tmp/ipykernel_217704/1044816083.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t1.authors.loc[t1.pkgid.isin([210011001, 210011002, 210011003, 210011004, 210287001])] = 'Peters, Debra; Huenneke, Laura'
/tmp/ipykernel_217704/1044816083.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t1.authors.loc[t1.pkgid.isin([210002001])] = 'Peters, Debra; Anderson, John; Huenneke, Laura'
/tmp/ipykernel_217704/1044816083.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#r

KeyError: '[210011001, 210013002] not found in axis'

In [15]:
t1_out = t1.loc[:, ['authors','pubdate','title','doi','corearea','drange',
                    'status','top10']]
t1_out.to_excel('tables_out/JRN8_SuppTable1_draft.xlsx', index=False)